In [1]:
## 3rd trial

In [1]:
import pandas as pd
import numpy as np
import scipy
from nltk.corpus import stopwords
import re
import os

In [2]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)

tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3] ## Remove unnecessary columns

tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case


tranzact_data3 = tranzact_data2.dropna() ## remove rows with cell value none

tranzact_data3 = tranzact_data3.reset_index(drop=True)

## Remove items sold by a company to its subsidiary :

drop_list = []

for i in range(len(tranzact_data3)):
    
    if (tranzact_data3['Supplier'][i].split())[0] == 'CESARE'and (tranzact_data3['Company'][i].split())[0] == 'CESARE':
        
        drop_list.append(i)
        
        tranzact_data = tranzact_data3.drop(drop_list)
                
tranzact_data = tranzact_data.reset_index(drop=True)

print(len(tranzact_data))
tranzact_data.head()       

24348


,Company,Supplier,Item
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."


In [3]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap

## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
words.append('min')
#words.append('machined')
words.append('top')
words.append('dia')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

print(len(tranzact_data))
print('================')
print(tranzact_data['features'][111])
print(type(tranzact_data['features'][111]))

## Removing duplicate words from individual features

## tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
##                                                                                key=x.split().index))]).lower())
print(len(tranzact_data))

## Removing only single word features :

tranzact_data = tranzact_data[tranzact_data['features'].str.contains(' ')]
tranzact_data = tranzact_data.reset_index(drop=True)

tranzact_data.tail()

24348
chromium vanadium spring steel rolled en47 wire rod
<class 'str'>
24348


,Company,Supplier,Item,features
22874,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
22875,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
22876,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
22877,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
22878,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [4]:
for i in range(len(tranzact_data)):
    if tranzact_data['features'][i] == '':
        
        print(i)
        print('===')
        tranzact_data['Supplier'][i] = np.nan
    
tranzact_data = tranzact_data.dropna()
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

22879


,Company,Supplier,Item,features
22874,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
22875,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
22876,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
22877,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
22878,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [5]:
## Removing duplicate features + Supplier combinations :

tranzact_data['find_duplicate'] = tranzact_data['Supplier'].map(str) + tranzact_data['features']
tranzact_data.drop_duplicates(subset='find_duplicate', keep = 'first', inplace = True)
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

16407


,Company,Supplier,Item,features,find_duplicate
16402,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11,M/S. S. K. Enterprisepvc gland pg11
16403,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13,M/S. S. K. Enterprisepvc gland pg13
16404,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1,M/S. S. K. Enterprisepvc gland m25x1
16405,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number,M/S. S. K. Enterpriseferrul number
16406,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long,Anant Tradershard copper pipe gauge long


In [6]:
## Removing unnecessary long features ( feature words > 25):

tranzact_data['number_of_words'] = tranzact_data.features.apply(lambda x: len(x.split()))

list_to_drop = []

for i in range(len(tranzact_data['number_of_words'])):
    
    if tranzact_data['number_of_words'][i] > 50 :
        
        list_to_drop.append(i)

tranzact_data_final = tranzact_data.drop(list_to_drop)
tranzact_data_final = tranzact_data_final.reset_index(drop=True)

print(max(tranzact_data_final['number_of_words']))

tranzact_data_final['word_increase'] = max(tranzact_data_final['number_of_words']) - tranzact_data_final['number_of_words']

tranzact_data_final.head()

49


,Company,Supplier,Item,features,find_duplicate,number_of_words,word_increase
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc,zincoplast zinc,Pukhraj Additives LLPzincoplast zinc,2,47
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn,zincoplast pyn,Pukhraj Additives LLPzincoplast pyn,2,47
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...,Bijal Steelschromium manganese spring steel ro...,8,41
3,Modern Engineering & Spring Company,Jay Khodiyar Steel & Metals,ms sheet 8 ft x 4 ft x 1.00 mm - 4 nos,ms sheet,Jay Khodiyar Steel & Metalsms sheet,2,47
4,Modern Engineering & Spring Company,Dynachem Engineers,ms plate 160 x 160 x 6 - 4 holes drg. z-413146...,ms plate holes drg rev ae0,Dynachem Engineersms plate holes drg rev ae0,6,43


In [89]:
for i in range(len(tranzact_data_final)):
    
    dict_for_df = {}
    
#    print('no. of word increase = ', tranzact_data_final['word_increase'][i])
    
    if tranzact_data_final['word_increase'][i] > 0:
        
        for j in range(tranzact_data_final['word_increase'][i]):
            
            repeat_word = 'ww '
            dict_for_df[j] = [repeat_word]
        
        df = pd.DataFrame(dict_for_df)
        
        df['extra'] = df.sum(axis = 1)
        
    
    
        tranzact_data_final['features'][i] = df['extra'][0] + tranzact_data_final['features'][i]
        df.drop('extra', axis=1, inplace=True)
    
#    print('row no. = ', i)
#    print(tranzact_data_final['features'][i])
#    print('==================================================')
    
    
#    print(tranzact_data_final['features'][i])
#    print(len(tranzact_data_final['features'][i].split()))


In [5]:
## To check no. of Suppliers & no. of unique featuresin the data base :

# print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data.features.value_counts()))
# print('Tranzact_data length = ', len(tranzact_data))
# tranzact_data.tail()

In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [8]:
## To cross check no. of Suppliers & no. of unique features in the data base :

# print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
# print('Total no. of entrys = ', len(tranzact_data_final))
# tranzact_data_final.tail()

In [7]:
# Creating seacrh engine file  and digitising original features :

from sklearn.feature_extraction.text import TfidfVectorizer

col = ['Supplier', 'features']
tranzact_data_search = tranzact_data_final[col].copy()


tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]  ## Indexing

supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
                                                        sort_values('Supplier_id')

## Create two dictionaries :

supplier_to_id = dict(supplier_id_tranzact_data_search.values)
id_to_supplier = dict(supplier_id_tranzact_data_search[['Supplier_id', 'Supplier']].values)

## Digitising :

print(len(tranzact_data_search['features']))
print(len(tranzact_data_search['features'][0]))
print(tranzact_data_search['features'][0])

my_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', ngram_range=(1, 2))
digital_features = my_vectorizer.fit(tranzact_data_search['features'])
my_vector = digital_features.transform(tranzact_data_search['features']).toarray()
print(my_vector.shape)
print(tranzact_data_search['features'][12490])
tranzact_data_search.tail()


16397
15
zincoplast zinc
(16397, 46852)
rm bnd close cover cast steel gr 520n


,Supplier,features,Supplier_id
16392,M/S. S. K. Enterprise,pvc gland pg11,309
16393,M/S. S. K. Enterprise,pvc gland pg13,309
16394,M/S. S. K. Enterprise,pvc gland m25x1,309
16395,M/S. S. K. Enterprise,ferrul number,309
16396,Anant Traders,hard copper pipe gauge long,2056


In [8]:
## Input Text data :

text_data = ['ceramic nozzle']

#text_data_lower_case = [x.lower() for x in text_data]


#text_data1 = pd.Series(text_data_lower_case).apply(lambda x:sorted(set(x.split())))


print(text_data)
print(type(text_data))

['ceramic nozzle']
<class 'list'>


In [9]:
## From Teaxt data to text feature :

text_data_lower_case = [x.lower() for x in text_data]

text_data1 = pd.Series(text_data_lower_case)
#print(type(text_data1))
#print('==================')

#text_data1 = pd.Series(text_data1.apply(lambda x:sorted(set(x.split()))))
#print(type(text_data1))

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
#words.append('machined')
words.append('top')
words.append('dia')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

# text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
#                                                                                key=x.split().index))]).lower())

print(text_data_feature)
print("=========")
print(type(text_data_feature))
print('==================')
print(len(tranzact_data_search['features']))
## Cosine Similarity :

def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []


merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))


print('==================')
print(len(tranzact_data_search['features']))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()

test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

print('%%%%%%%%%%%%%%%%%%%%%%%%%%')
print(merge_text_feature.shape)
print('===================')
print(merge_digit_vector.shape)
print('===================')
print(type(my_vector))
print(my_vector.shape)
print('===================')
print(type(test_digit_vector))
print(test_digit_vector.shape)
print('%%%%%%%%%%%%%%%%%%%%%%%%%')


## Finding cosine similarity :

c_d = []

# c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)
c_d = np.round((1 - cos_cdist(merge_digit_vector[0:len(merge_digit_vector)-2], test_digit_vector)), 3)
print(type(c_d))
print(c_d.shape)
print(text_data_feature)
print('+++++++++++')
print(len(tranzact_data_search))
#merge_text_feature.tail()

0    ceramic nozzle
dtype: object
<class 'pandas.core.series.Series'>
16397
16397
%%%%%%%%%%%%%%%%%%%%%%%%%%
(16398,)
(16398, 46852)
<class 'numpy.ndarray'>
(16397, 46852)
<class 'numpy.ndarray'>
(46852,)
%%%%%%%%%%%%%%%%%%%%%%%%%
<class 'numpy.ndarray'>
(16396,)
0    ceramic nozzle
dtype: object
+++++++++++
16397


In [11]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]
#max_c_s_index = np.argsort(aa_sum)[-50:][::-1]

for i in max_c_s_index:
    
      
    result.append(np.array([c_d[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
                            tranzact_data_search['Supplier'][i]]))

        
    
#    result.append(np.array([aa_sum[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
#                            tranzact_data_search['Supplier'][i]]))


result_frame = pd.DataFrame(result, columns=['c_index', 'Item', 'features', 'Supplier'])


result_frame.c_index = result_frame.c_index.astype('float64')

result_frame.drop_duplicates(subset='Supplier', keep = 'first', inplace = True)
result_frame = result_frame.reset_index(drop=True)

kount = 0

for i in range(len(result_frame['Supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['Supplier'][i])
        print(i)
        kount = kount+1
        if kount == 5 :
            break
result_frame.head(10)

Hari Engineering
0
BHAGDEV WELDING TRADERS
1
YOGEETA ENTERPRISES
2
SAP INDUSTRIES
3
Technocrats Plasma Systems Pvt. Ltd.
4


,c_index,Item,features,Supplier
0,1.000,ceramic nozzle,ceramic nozzle,Hari Engineering
1,0.786,ceramic nozzle ( size 10 ),ceramic nozzle size,BHAGDEV WELDING TRADERS
2,0.786,ceramic nozzle size 8,ceramic nozzle size,YOGEETA ENTERPRISES
3,0.786,ceramic nozzle size 6,ceramic nozzle size,SAP INDUSTRIES
4,0.311,nozzle holder,nozzle holder,Technocrats Plasma Systems Pvt. Ltd.
5,0.310,s.s.304 spray nozzle,spray nozzle,Spraytech
6,0.305,consumables - nozzle,consumables nozzle,MODERN EQUIPMENTS & SALES PVT. LTD.
7,0.248,welding nozzle / cap,welding nozzle cap,Pavan Traders
8,0.244,cutting nozzle b-type 1/16,cutting nozzle type,Manish International
9,0.243,"hardware - cutting nozzle - 3/64""",hardware cutting nozzle,PANTH ENTERPRISES


In [19]:
for j in range(10):
    
    if result_frame['features'][j] != result_frame['features'][j+1]:
        
        text_ata_1 = [result_frame['features'][j+1] + ' ' + result_frame['features'][j]]
                                        

print(text_data_1)
print(type(text_data_1))

['ceramic nozzle size ceramic nozzle']
<class 'list'>


In [70]:
dict_for_df = {}
for i in ('a','b','c','d'):    # Don't use "id" as a counter; it's a python function
    x = 3        # first value
    y = 5        # second value
    dict_for_df[i] = [x,y]     # store in a dict
df = pd.DataFrame(dict_for_df) # after the loop convert the dict to a dataframe


In [71]:
df.head()

,a,b,c,d
0,3,3,3,3
1,5,5,5,5


In [76]:
dict_for_df = {}
for i in range(6):    # Don't use "id" as a counter; it's a python function
    x = 'xx'        # first value
    y = 'yy'        # second value
    z = 'zz'
    dict_for_df[i] = [x,y,z]     # store in a dict
df = pd.DataFrame(dict_for_df) # after the loop convert the dict to a dataframe
df.head()

,0,1,2,3,4,5
0,xx,xx,xx,xx,xx,xx
1,yy,yy,yy,yy,yy,yy
2,zz,zz,zz,zz,zz,zz


In [46]:
dict_for_df = {}
for i in range(6): 
    
    x = 'xx '                 
    dict_for_df[i] = [x]     # store in a dict
df = pd.DataFrame(dict_for_df) # after the loop convert the dict to a dataframe
print(df)

df['extra'] = df.sum(axis = 1)

print(df)


     0    1    2    3    4    5
0  xx   xx   xx   xx   xx   xx 
     0    1    2    3    4    5               extra
0  xx   xx   xx   xx   xx   xx   xx xx xx xx xx xx 


In [66]:

for i in range(len(tranzact_data_final)):
    
    dict_for_df = {}
    
    print('no. of word increase = ', tranzact_data_final['word_increase'][i])
    
    if tranzact_data_final['word_increase'][i] > 0:
        
        for j in range(tranzact_data_final['word_increase'][i]+1):
            
            repeat_word = 'ww '
            dict_for_df[j] = [repeat_word]
        
        df = pd.DataFrame(dict_for_df)
        
        df['extra'] = df.sum(axis = 1)
        
    print(df)
    
    tranzact_data_final['features'][i] = df['extra'][0] + tranzact_data_final['features'][i]
    df.drop('extra', axis=1, inplace=True)
    print('==========================================================')
    break
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')        
print(tranzact_data_final['features'][0])
df.info()

print(len(tranzact_data_final['features'][0].split()))
#    pd.Series(tranzact_data_final['feature_equal'][i]) = pd.Series(df['extra'][0]) + \
#                                                         pd.Series(tranzact_data_final['features'][i]) 
    
    
    
#    aaa[i] = pd.DataFrame({})
#    for j in range(tranzact_data_final['word_increase'][i]+1):
            
#        aaa[i] = aaa[i] + df[j].map(str)+ ' '
#    print(aaa[1])
#    break    
    
        
            
#            if j == tranzact_data_final['word_increase'][i] - 1:
#                tranzact_data_final['how'][i] = aaa + tranzact_data_final['features'][i]
                

no. of word increase =  43
     0    1    2    3    4    5    6    7    8    9  \
0  ww   ww   ww   ww   ww   ww   ww   ww   ww   ww    

                         ...                           35   36   37   38   39  \
0                        ...                          ww   ww   ww   ww   ww    

    40   41   42   43                                              extra  
0  ww   ww   ww   ww   ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww w...  

[1 rows x 45 columns]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww ww

In [103]:
print(len(aaa))
print(aaa[10000])
df.head()

16373
0    NaN
dtype: object


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,ww,ww,ww,ww,ww,ww,ww,ww,ww,ww,...,ww,ww,ww,ww,ww,ww,ww,ww,ww,ww


In [97]:
tranzact_data_final['new_features'] = aaa + tranzact_data_final['features']

In [100]:
col_1 = ['features', 'Supplier', 'new_features']
ddf = tranzact_data_final[col_1].copy()
ddf.head()

,features,Supplier,new_features
0,zincoplast zinc,Pukhraj Additives LLP,0 NaN dtype: object
1,zincoplast pyn,Pukhraj Additives LLP,0 NaN dtype: object
2,chromium manganese spring steel rolled sup9 wi...,Bijal Steels,0 NaN dtype: object
3,ms sheet,Jay Khodiyar Steel & Metals,0 NaN dtype: object
4,ms plate holes drg rev ae0,Dynachem Engineers,0 NaN dtype: object
